In [1]:
import numpy as np
from random import gauss
from math import sqrt
import matplotlib as mpl
mpl.use('nbagg')
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

In [2]:
def TE(x, y, xhist_list, yhist_list):
    Information = dict()

    parms = dict()
    parms['max_iters'] = 20
    # shifted data
    # shift>0 => y shifted to positive side
    xedge = np.append(np.unique(x),(max(x)+1))
    yedge = np.append(np.unique(y),(max(y)+1))
    dat = []
    edges = []
    end = min(xhist_list[-1], yhist_list[-1])
    for hist in xhist_list:
        xhist = x[-end+hist:hist]
        dat.append(xhist.copy())
        edges.append(xedge)
    for hist in yhist_list:
        yhist = y[-end+hist:hist]
        dat.append(yhist.copy())
        edges.append(yedge)
    dat.append(y[-end:])
    edges.append(yedge)
    N, _ = np.histogramdd(np.array(dat).T, bins=edges)
    N = np.reshape(N, (len(np.unique(x))**len(xhist_list),len(np.unique(y))**len(yhist_list), len(np.unique(y))))
    #Calculate all kinds of probability and make sure the shape of them, 0 -> x, 1 -> v, 2 -> r
    # x repersent x-hist
    # v repersent y-hist
    # r repersent y-now
    px=(np.sum(N,axis=(1,2))/np.sum(N))[:, np.newaxis, np.newaxis]
    pv=(np.sum(N,axis=(0,2))/np.sum(N))[np.newaxis, :, np.newaxis]
    pr=(np.sum(N,axis=(0,1))/np.sum(N))[np.newaxis ,np.newaxis, :]
    pxv=(np.sum(N,axis=2)/np.sum(N))[:, :, np.newaxis]
    pxr=(np.sum(N,axis=1)/np.sum(N))[:, np.newaxis, :]
    pvr=(np.sum(N,axis=0)/np.sum(N))[np.newaxis, :, :]
    pxvr=(N/np.sum(N))


    MIxr=np.nansum(pxr*np.log2(pxr/px/pr))
    MIvr=np.nansum(pvr*np.log2(pvr/pv/pr))
    MIxvR=np.nansum(pxvr*np.log2(pxvr/pxv/pr))
    PI_xR = np.nansum(pxr*np.log2(pxr/px/pr), axis = (0,1))
    PI_vR = np.nansum(pvr*np.log2(pvr/pv/pr), axis = (0,1))
    R = sum(np.minimum(PI_xR, PI_vR))
    Information[('Beer','Red')] = R
    Information[('Beer','UIx2y')] = MIxr - R
    Information[('Beer','UIy2y')] = MIvr - R
    Information[('Beer','Syn')] = MIxvR - MIxr - MIvr + R

    return Information